## Tfidftransformer Usage

### Dataset and Imports

In [11]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 

In [5]:
import tensorflow as tf

In [13]:
docs=["the house had a tiny little mouse", 
"the cat saw the mouse", 
"the mouse ran away from the house", 
"the cat finally ate the mouse", 
"the end of the mouse story"
     ]

### Initialize CountVectorizer

In [15]:
#instantiate CountVectorizer() 
cv = CountVectorizer() 

In [16]:
# 문서에 있는 단어에 대한 단어 수 생성
word_count_vector=cv.fit_transform(docs)

In [17]:
word_count_vector.shape

(5, 16)

* 이때, 임의로 불용어 리스트를 지정하고 최소 단어 수를 적용하는 작업을 수행할 수 있다. 사용방법은 문서 참조
<br>

https://kavita-ganesan.com/how-to-use-countvectorizer/#.YZd6ed_P2Uk

### Compute the IDF values

In [19]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [20]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
mouse,1.000000
the,1.000000
cat,1.693147
house,1.693147
ate,2.098612
away,2.098612
end,2.098612
finally,2.098612
from,2.098612
had,2.098612


* mouse 및 the 단어들은 모든 문서에 나타나기 때문에 해당 단어들의 IDF 값이 가장 낮다,
* In practice, your IDF should be based on a large corpora of text.

### Compute the TFIDF score for your documents

IDF 값을 계산하고 나면, 이제 문서나 문서 집합에 대한 tf-idf 점수를 계산할 수 있게 된다.

In [21]:
# 해당 5개의 문서 에 대한 tf-idf 점수를 계산한다.
# count matrix 
count_vector=cv.transform(docs) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

위의 첫 번째 줄은 희소 행렬 형식의 문서에 대한 단어 수를 가져온다.<br>
새로운 문서 집합에서 tf-idf 점수를 계산하기 위해서<br> 먼저 cv.transfrom(new_docs)를 word counts의 행렬로 생성해야 한다.

 tfidf_transformer.transform(count_vector)을 실행함으로써 문서에 대한 tf-idf 점수를 계산할 수 있게 된다. 

### 첫 번째 문서의 tf-idf 점수를 데이터 프레임에 넣고 내림차순으로 정렬

In [23]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
had,0.493562
little,0.493562
tiny,0.493562
house,0.398203
mouse,0.235185
the,0.235185
ate,0.000000
away,0.000000
cat,0.000000
end,0.000000


여러 문서에서 공통적으로 사용되는 단어일수록 점수가 낮고 첫 번째 문서 에서 고유한 단어일수록 점수가 높다

## Tfidfvectorizer Usage

Tfidfvectorizer 를 사용하면 단어 수, idf 및 tf-idf 값을 한 번에 계산한다. 

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
# count vectorizer 에서 사용하는 설정이 이동
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
 
# 사용했던 문서를 그대로 가져오기 (docs) 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)

In [26]:
# 첫 번째 벡터 가져오기
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 
 
# 데이터 프레임에 tf-idf 값을 나열
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)


,tfidf
had,0.493562
little,0.493562
tiny,0.493562
house,0.398203
mouse,0.235185
the,0.235185
ate,0.000000
away,0.000000
cat,0.000000
end,0.000000


### 또 다른 방법

In [27]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
### 
fitted_vectorizer=tfidf_vectorizer.fit(docs)
tfidf_vectorizer_vectors=fitted_vectorizer.transform(docs)

---

## Tfidftransformer vs. Tfidfvectorizer

> Tfidftransformer에서는 <br>
CountVectorizer를 사용하여 체계적으로 word_count를 계산한 다음 <br>IDF (Inverse Document Frequenct) 값을 계산하고 TF-IDF 점수만 계산한다.

> Tfidfvectorizer에서는 <br>
반대로 세 단계를 모두 한 번에 수행한다.<br>
동일한 데이터 세트를 사용하여 단어 수, IDF 값 및 TF-IDF 점수를 계산한다

### 결론
* 서로 다른 작업에 대해 빈도(term count) 벡터가 필요하다면 Tfidftransformer를 사용
* 학습 (training) 데이터 세트 내의 문서에 대한 TF-idf 점수를 계산해야 하는 경우 Tfidfvectorizer를 사용
* 학습 (training) 데이터 세트 외부에 있는 문서에서 tf-idf 점수를 계산해야 하는 경우 둘 중 하나를 사용하면 모두 작동한다.